In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

import pandas as pd

# Daten aus der CSV-Datei lesen
df = pd.read_csv('audio_data.csv')

# Aufteilen der Daten in Features und Labels
X = df[['mfcc', 'mel']].values
y = df['label'].values

# Flachlegen der 'mfcc' und 'mel' Daten
X_mfcc = np.array(df['mfcc'].to_list())
X_mel = np.array(df['mel'].to_list())

# Zusammenführen der Daten
X = np.concatenate([X_mfcc, X_mel], axis=0)

# Konvertieren der Daten in PyTorch-Tensoren
X_train = torch.from_numpy(X).float()
y_train = torch.from_numpy(y).long()

# Erstellen des CNN-Modells
print("defining model")
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv(1, 32, (3, 3))
        self.pool1 = nn.MaxPooling2d((2, 2))
        self.conv2 = nn.Conv(32, 64, (3, 3))
        self.pool2 = nn.MaxPooling2d((2, 2))
        self.fc1 = nn.Linear(64 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

# Initialisieren des Modells
model = CNN()

# Definieren des Optimierers und der Verlustfunktion
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

print("training model")
# Trainieren des Modells
for epoch in range(10):
    # Trainieren des Modells mit einem Batch
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Drucken des Verlusts
    print(f"Epoch {epoch + 1}: {loss.item()}")


# # Bewertung des Modells
# with torch.no_grad():
#     outputs = model(X_test)
#     _, predicted = torch.max(outputs.data, 1)
#     accuracy = (predicted == y_test).sum().item() / len(y_test)

# print(f"Accuracy: {accuracy}")

# Speichern des Modells
torch.save(model.state_dict(), "model.pt")


TypeError: can't convert np.ndarray of type numpy.str_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.